In [ ]:
"""
Created on Fri May 21 14:29 2021

Apply tuned parameters to the params

@author: Clara Burgard
"""

In [ ]:
import xarray as xr
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import cm
import cartopy
import cartopy.crs as ccrs
import matplotlib as mpl
import pandas as pd 
from tqdm.notebook import trange, tqdm
import time
import basal_melt_param.melt_functions as meltf
import basal_melt_param.plume_functions as pf
import basal_melt_param.analysis_functions as ana
import basal_melt_param.useful_functions as uf
from basal_melt_param.constants import *
import seaborn as sns
import os, glob
import distributed

In [ ]:
print(mpl.get_backend())

In [ ]:
%matplotlib qt5

READ IN DATA

In [ ]:
nemo_run = 'bi646'

In [ ]:
inputpath_data='/bettik/burgardc/DATA/NN_PARAM/interim/SMITH_'+nemo_run+'/'
inputpath_mask = '/bettik/burgardc/DATA/NN_PARAM/interim/ANTARCTICA_IS_MASKS/SMITH_'+nemo_run+'/'
inputpath_profiles = '/bettik/burgardc/DATA/NN_PARAM/interim/T_S_PROF/SMITH_'+nemo_run+'/'
outputpath_simple_all = '/bettik/burgardc/DATA/BASAL_MELT_PARAM/interim/SIMPLE/nemo_5km_06161821_oneFRIS/'
inputpath_plumes = '/bettik/burgardc/DATA/NN_PARAM/interim/PLUMES/SMITH_'+nemo_run+'/'
outputpath_plumes = '/bettik/burgardc/DATA/NN_PARAM/interim/PLUMES/SMITH_'+nemo_run+'/'
inputpath_boxes = '/bettik/burgardc/DATA/NN_PARAM/interim/BOXES/SMITH_'+nemo_run+'/'
outputpath_boxes = '/bettik/burgardc/DATA/NN_PARAM/interim/BOXES/SMITH_'+nemo_run+'/'
outputpath_melt = '/bettik/burgardc/DATA/NN_PARAM/processed/MELT_RATE/SMITH_'+nemo_run+'_CLASSIC/'
outputpath = '/bettik/burgardc/DATA/NN_PARAM/interim/'

In [ ]:
# make the domain a little smaller to make the computation even more efficient - file isf has already been made smaller at its creation
map_lim = [-3000000,3000000]

In [ ]:
file_other = xr.open_dataset(inputpath_data+'corrected_draft_bathy_isf.nc')#, chunks={'x': chunk_size, 'y': chunk_size})
file_other_cut = uf.cut_domain_stereo(file_other, map_lim, map_lim)
#file_other_cut = file_other_cut.assign_coords({'time': range(1970,len(file_other_cut.time)+1970)})

file_conc = xr.open_dataset(inputpath_data+'isfdraft_conc_Ant_stereo.nc')
file_conc_cut = uf.cut_domain_stereo(file_conc, map_lim, map_lim)
#file_conc_cut = file_conc_cut.assign_coords({'time': range(1970,len(file_conc_cut.time)+1970)})

In [ ]:
# SIMPLE PARAMETERS
gammas_file_simple = xr.open_dataset(outputpath_simple_all+'gammas_simple_ALL.nc')
#simple_param_types = ['quadratic_local', 'quadratic_local_locslope'] 
simple_param_types = ['linear_local']


# PLUME PARAMETERS
plume_param_types = ['lazero19_2'] #'lazero19_2',

tuned_plume_gammas = xr.DataArray(np.array([[2.8e-4,42.2e-4],
                                              [1.3e-4,1e3]]), dims={'param': ['lazero19_2','lazero19_modif2'], 'profile_domain': np.array([50,1000])})
tuned_plume_gammas = tuned_plume_gammas.assign_coords({'param': ['lazero19_2','lazero19_modif2'], 'profile_domain': np.array([50,1000])})

tuned_plume_E0s = xr.DataArray(np.array([[4.2e-2,0.34e-2],
                                  [7.6e-2,0.14e-2]]), dims={'param': ['lazero19_2','lazero19_modif2'], 'profile_domain': np.array([50,1000])})
tuned_plume_E0s = tuned_plume_E0s.assign_coords({'param': ['lazero19_2','lazero19_modif2'], 'profile_domain': np.array([50,1000])})

# BOX PARAMETERS

pism_version = 'yes'
picop_opt = 'no'

tuned_box_gammas = xr.DataArray(np.array([[[0.39e-5,0.41e-5,0.44e-5,0.39e-5],
                                              [0.39e-5,0.41e-5,0.44e-5,0.39e-5]],
                                         [[0.51e-5,0.73e-5,0.92e-5,0.63e-5],
                                              [0.51e-5,0.73e-5,0.92e-5,0.63e-5]]]
                                        ), dims={'profile_domain': np.array([50,1000]), 'pism': ['no','yes'], 'config': range(1,5)})
tuned_box_gammas = tuned_box_gammas.assign_coords({'profile_domain': np.array([50,1000]),'pism': ['no','yes'], 'config': range(1,5)})


tuned_box_Cs = xr.DataArray(np.array([[[16.5e6,18.0e6,20.7e6,16.1e6],
                                              [16.1e6,17.8e6,20.5e6,20.5e6]],
                                    [[0.14e6,0.14e6,0.14e6,0.13e6],
                                              [0.14e6,0.14e6,0.14e6,0.13e6]]]), dims={'profile_domain': np.array([50,1000]), 'pism': ['no','yes'], 'config': range(1,5)})
tuned_box_Cs = tuned_box_Cs.assign_coords({'profile_domain': np.array([50,1000]),'pism': ['no','yes'], 'config': range(1,5)})

In [ ]:
for yyear in range(1980+50,1980+60):
    print(yyear)
    
    # PREPARATION
    file_isf_orig = xr.open_dataset(inputpath_mask+'nemo_5km_isf_masks_and_info_and_distance_oneFRIS_'+str(yyear)+'.nc').drop('time')
    nonnan_Nisf = file_isf_orig['Nisf'].where(np.isfinite(file_isf_orig['front_bot_depth_max']), drop=True).astype(int)
    file_isf_nonnan = file_isf_orig.sel(Nisf=nonnan_Nisf)
    large_isf = file_isf_nonnan['Nisf'].where(file_isf_nonnan['isf_area_here'] >= 2500, drop=True)
    file_isf = file_isf_nonnan.sel(Nisf=large_isf)
    file_isf['isf_name'] = file_isf['isf_name'].astype(str)
    file_isf['region'] = file_isf['region'].astype(str)
    
    lon = file_isf.longitude
    lat = file_isf.latitude
    
    file_isf_conc = file_conc_cut['isfdraft_conc'].sel(time=yyear).drop('time')

    xx = file_isf.x
    yy = file_isf.y
    dx = (xx[2] - xx[1]).values
    dy = (yy[2] - yy[1]).values
    grid_cell_area = abs(dx*dy)  
    grid_cell_area_weighted = file_isf_conc * grid_cell_area
    
    ice_draft_pos = file_other_cut['corrected_isfdraft'].sel(time=yyear).drop('time')
    ice_draft_neg = -ice_draft_pos

    file_TS_orig = xr.open_dataset(inputpath_profiles+'T_S_mean_prof_corrected_km_contshelf_'+str(yyear)+'.nc')
    file_TS = file_TS_orig.sel(Nisf=large_isf)
    
    isf_stack_mask = uf.create_stacked_mask(file_isf['ISF_mask'], file_isf.Nisf, ['y','x'], 'mask_coord')
    
    box_charac_all_2D = xr.open_dataset(inputpath_boxes + 'nemo_5km_boxes_2D_oneFRIS_'+str(yyear)+'_merged75.nc')
    box_charac_all_1D = xr.open_dataset(inputpath_boxes + 'nemo_5km_boxes_1D_oneFRIS_'+str(yyear)+'_merged75.nc')
    plume_charac = xr.open_dataset(inputpath_plumes+'nemo_5km_plume_characteristics_oneFRIS_'+str(yyear)+'_merged75.nc')
    
    param_var_of_int_2D = file_isf[['ISF_mask', 'latitude', 'longitude', 'dGL']]
    param_var_of_int_1D = file_isf[['front_bot_depth_avg', 'front_bot_depth_max','isf_name']]
    
    geometry_info_2D = plume_charac.merge(param_var_of_int_2D).merge(ice_draft_pos).rename({'corrected_isfdraft':'ice_draft_pos'}).merge(grid_cell_area_weighted).rename({'isfdraft_conc':'grid_cell_area_weighted'}).merge(file_isf_conc)
    geometry_info_1D = param_var_of_int_1D

    nisf_list = geometry_info_1D.Nisf
    T_S_profile = file_TS.ffill(dim='depth').squeeze().drop('time')
    
    # SIMPLE PARAMS
    for mparam in simple_param_types:

        gamma = gammas_file_simple['slope'].sel(param=mparam,profile_domain=50)
        ds_2D, ds_1D = meltf.calculate_melt_rate_1D_and_2D_all_isf(nisf_list, T_S_profile, geometry_info_2D, geometry_info_1D, isf_stack_mask, mparam, gamma, U_param=True, 
                                                                   options_2D=['melt_m_ice_per_y'], options_1D=['melt_Gt_per_y_tot'], verbose=False)

        ds_2D.to_netcdf(outputpath_melt+'melt_rates_2D_'+mparam+'_correctedTS_oneFRIS_'+str(yyear)+'.nc')
        ds_1D.to_netcdf(outputpath_melt+'melt_rates_1D_'+mparam+'_correctedTS_oneFRIS_'+str(yyear)+'.nc')
        
    for mparam in plume_param_types:

        gamma = tuned_plume_gammas.sel(param=mparam, profile_domain=50)
        E0 = tuned_plume_E0s.sel(param=mparam, profile_domain=50)

        ds_2D, ds_1D = meltf.calculate_melt_rate_1D_and_2D_all_isf(nisf_list, T_S_profile, geometry_info_2D, geometry_info_1D, isf_stack_mask, mparam, gamma, E0=E0, options_2D=['melt_m_ice_per_y'], 
                                                                   options_1D=['melt_Gt_per_y_tot'], verbose=False)
                           
        ds_2D.to_netcdf(outputpath_melt+'melt_rates_2D_'+mparam+'_correctedTS_oneFRIS_'+str(yyear)+'.nc')
        ds_1D.to_netcdf(outputpath_melt+'melt_rates_1D_'+mparam+'_correctedTS_oneFRIS_'+str(yyear)+'.nc')

    for nD_config in range(4,5):
        
        mparam = 'boxes_'+str(nD_config)+'_pism'+pism_version+'_picop'+picop_opt
              
        C = tuned_box_Cs.sel(config=nD_config, pism=pism_version, profile_domain=50).drop('config').drop('pism').drop('profile_domain')
        gamma = tuned_box_gammas.sel(config=nD_config, pism=pism_version, profile_domain=50).drop('config').drop('pism').drop('profile_domain')
                    
                        
        ds_2D, ds_1D = meltf.calculate_melt_rate_1D_and_2D_all_isf(nisf_list, T_S_profile, geometry_info_2D, geometry_info_1D, isf_stack_mask, mparam, gamma,
                      C=C, angle_option='appenB', box_charac_2D=box_charac_all_2D, box_charac_1D=box_charac_all_1D, box_tot=nD_config, box_tot_option='nD_config', 
                      pism_version=pism_version, picop_opt=picop_opt, options_2D=['melt_m_ice_per_y'],options_1D=['melt_Gt_per_y_tot'], verbose=False)

                    
        ds_2D.to_netcdf(outputpath_melt+'melt_rates_2D_'+mparam+'_correctedTS_oneFRIS_'+str(yyear)+'.nc')
        ds_1D.to_netcdf(outputpath_melt+'melt_rates_1D_'+mparam+'_correctedTS_oneFRIS_'+str(yyear)+'.nc')

EVALMETRICS ONLY

In [ ]:
for yyear in tqdm(range(1970, 2051)):
    #print(yyear)
    
    # PREPARATION
    file_isf_orig = xr.open_dataset(inputpath_mask+'nemo_5km_isf_masks_and_info_and_distance_oneFRIS_'+str(yyear)+'.nc').drop('time')
    nonnan_Nisf = file_isf_orig['Nisf'].where(np.isfinite(file_isf_orig['front_bot_depth_max']), drop=True).astype(int)
    file_isf_nonnan = file_isf_orig.sel(Nisf=nonnan_Nisf)
    large_isf = file_isf_nonnan['Nisf'].where(file_isf_nonnan['isf_area_here'] >= 2500, drop=True)
    file_isf = file_isf_nonnan.sel(Nisf=large_isf)
    file_isf['isf_name'] = file_isf['isf_name'].astype(str)
    file_isf['region'] = file_isf['region'].astype(str)
    
    lon = file_isf.longitude
    lat = file_isf.latitude
    
    file_isf_conc = file_conc_cut['isfdraft_conc'].sel(time=yyear).drop('time')

    xx = file_isf.x
    yy = file_isf.y
    dx = (xx[2] - xx[1]).values
    dy = (yy[2] - yy[1]).values
    grid_cell_area = abs(dx*dy)  
    grid_cell_area_weighted = file_isf_conc * grid_cell_area
    
    ice_draft_pos = file_other_cut['corrected_isfdraft'].sel(time=yyear).drop('time')
    ice_draft_neg = -ice_draft_pos

    file_TS_orig = xr.open_dataset(inputpath_profiles+'T_S_mean_prof_corrected_km_contshelf_'+str(yyear)+'.nc')
    file_TS = file_TS_orig.sel(Nisf=large_isf)
    
    isf_stack_mask = uf.create_stacked_mask(file_isf['ISF_mask'], file_isf.Nisf, ['y','x'], 'mask_coord')
    
    box_charac_all_2D = xr.open_dataset(inputpath_boxes + 'nemo_5km_boxes_2D_oneFRIS_'+str(yyear)+'_merged75.nc')
    box_charac_all_1D = xr.open_dataset(inputpath_boxes + 'nemo_5km_boxes_1D_oneFRIS_'+str(yyear)+'_merged75.nc')
    plume_charac = xr.open_dataset(inputpath_plumes+'nemo_5km_plume_characteristics_oneFRIS_'+str(yyear)+'_merged75.nc')
    
    param_var_of_int_2D = file_isf[['ISF_mask', 'latitude', 'longitude', 'dGL']]
    param_var_of_int_1D = file_isf[['front_bot_depth_avg', 'front_bot_depth_max','isf_name']]
    
    geometry_info_2D = plume_charac.merge(param_var_of_int_2D).merge(ice_draft_pos).rename({'corrected_isfdraft':'ice_draft_pos'}).merge(grid_cell_area_weighted).rename({'isfdraft_conc':'grid_cell_area_weighted'}).merge(file_isf_conc)
    geometry_info_1D = param_var_of_int_1D

    nisf_list = geometry_info_1D.Nisf
    T_S_profile = file_TS.ffill(dim='depth').squeeze().drop('time')
    
    # SIMPLE PARAMS
    for mparam in simple_param_types:

        gamma = gammas_file_simple['slope'].sel(param=mparam,profile_domain=50)
        
        eval_1D = meltf.calculate_melt_rate_Gt_and_box1_all_isf(nisf_list, T_S_profile, 
                                                                geometry_info_2D, geometry_info_1D, isf_stack_mask, 
                                                                mparam, gamma, U_param=True, 
                                                                box_charac_2D=box_charac_all_2D, box_charac_1D=box_charac_all_1D, verbose=False)
        
        eval_1D.to_netcdf(outputpath_melt+'eval_metrics_1D_'+mparam+'_oneFRIS_'+str(yyear)+'.nc')    

In [ ]:
for yyear in tqdm(range(1970, 2051)):
    #print(yyear)
    
    # PREPARATION
    file_isf_orig = xr.open_dataset(inputpath_mask+'nemo_5km_isf_masks_and_info_and_distance_oneFRIS_'+str(yyear)+'.nc').drop('time')
    nonnan_Nisf = file_isf_orig['Nisf'].where(np.isfinite(file_isf_orig['front_bot_depth_max']), drop=True).astype(int)
    file_isf_nonnan = file_isf_orig.sel(Nisf=nonnan_Nisf)
    large_isf = file_isf_nonnan['Nisf'].where(file_isf_nonnan['isf_area_here'] >= 2500, drop=True)
    file_isf = file_isf_nonnan.sel(Nisf=large_isf)
    file_isf['isf_name'] = file_isf['isf_name'].astype(str)
    file_isf['region'] = file_isf['region'].astype(str)
    
    lon = file_isf.longitude
    lat = file_isf.latitude
    
    file_isf_conc = file_conc_cut['isfdraft_conc'].sel(time=yyear).drop('time')

    xx = file_isf.x
    yy = file_isf.y
    dx = (xx[2] - xx[1]).values
    dy = (yy[2] - yy[1]).values
    grid_cell_area = abs(dx*dy)  
    grid_cell_area_weighted = file_isf_conc * grid_cell_area
    
    ice_draft_pos = file_other_cut['corrected_isfdraft'].sel(time=yyear).drop('time')
    ice_draft_neg = -ice_draft_pos

    file_TS_orig = xr.open_dataset(inputpath_profiles+'T_S_mean_prof_corrected_km_contshelf_'+str(yyear)+'.nc')
    file_TS = file_TS_orig.sel(Nisf=large_isf)
    
    isf_stack_mask = uf.create_stacked_mask(file_isf['ISF_mask'], file_isf.Nisf, ['y','x'], 'mask_coord')
    
    box_charac_all_2D = xr.open_dataset(inputpath_boxes + 'nemo_5km_boxes_2D_oneFRIS_'+str(yyear)+'_merged75.nc')
    box_charac_all_1D = xr.open_dataset(inputpath_boxes + 'nemo_5km_boxes_1D_oneFRIS_'+str(yyear)+'_merged75.nc')
    plume_charac = xr.open_dataset(inputpath_plumes+'nemo_5km_plume_characteristics_oneFRIS_'+str(yyear)+'_merged75.nc')
    
    param_var_of_int_2D = file_isf[['ISF_mask', 'latitude', 'longitude', 'dGL']]
    param_var_of_int_1D = file_isf[['front_bot_depth_avg', 'front_bot_depth_max','isf_name']]
    
    geometry_info_2D = plume_charac.merge(param_var_of_int_2D).merge(ice_draft_pos).rename({'corrected_isfdraft':'ice_draft_pos'}).merge(grid_cell_area_weighted).rename({'isfdraft_conc':'grid_cell_area_weighted'}).merge(file_isf_conc)
    geometry_info_1D = param_var_of_int_1D

    nisf_list = geometry_info_1D.Nisf
    T_S_profile = file_TS.ffill(dim='depth').squeeze().drop('time')
    
    # SIMPLE PARAMS
    for mparam in simple_param_types:

        gamma = gammas_file_simple['slope'].sel(param=mparam,profile_domain=50)
        
        eval_1D = meltf.calculate_melt_rate_Gt_and_box1_all_isf(nisf_list, T_S_profile, 
                                                                geometry_info_2D, geometry_info_1D, isf_stack_mask, 
                                                                mparam, gamma, U_param=True, 
                                                                box_charac_2D=box_charac_all_2D, box_charac_1D=box_charac_all_1D, verbose=False)
        
        eval_1D.to_netcdf(outputpath_melt+'eval_metrics_1D_'+mparam+'_oneFRIS_'+str(yyear)+'.nc')        
        
    for mparam in plume_param_types:

        gamma = tuned_plume_gammas.sel(param=mparam, profile_domain=50)
        E0 = tuned_plume_E0s.sel(param=mparam, profile_domain=50)

        eval_1D = meltf.calculate_melt_rate_Gt_and_box1_all_isf(nisf_list, T_S_profile, 
                                                             geometry_info_2D, geometry_info_1D, isf_stack_mask, 
                                                             mparam, gamma, E0=E0, 
                                                             box_charac_2D=box_charac_all_2D, box_charac_1D=box_charac_all_1D, verbose=False)   
        
        eval_1D.to_netcdf(outputpath_melt+'eval_metrics_1D_'+mparam+'_oneFRIS_'+str(yyear)+'.nc')        

    for nD_config in range(3,5):
        
        pism_version = 'yes'
        picop_opt = 'no'
        mparam = 'boxes_'+str(nD_config)+'_pism'+pism_version+'_picop'+picop_opt
              
        C = tuned_box_Cs.sel(config=nD_config, pism=pism_version, profile_domain=50).drop('config').drop('pism').drop('profile_domain')
        gamma = tuned_box_gammas.sel(config=nD_config, pism=pism_version, profile_domain=50).drop('config').drop('pism').drop('profile_domain')
                    
                        
        eval_1D = meltf.calculate_melt_rate_Gt_and_box1_all_isf(nisf_list, T_S_profile, 
                                                                 geometry_info_2D, geometry_info_1D, isf_stack_mask, 
                                                                 mparam, gamma, C=C, angle_option='appenB',
                                                                 box_charac_2D=box_charac_all_2D, box_charac_1D=box_charac_all_1D, box_tot=nD_config, 
                                                                 box_tot_option='nD_config', pism_version=pism_version, picop_opt='no',
                                                                 verbose=False)
        
        eval_1D.to_netcdf(outputpath_melt+'eval_metrics_1D_'+mparam+'_oneFRIS_'+str(yyear)+'.nc')        


    for nD_config in range(4,5):
        
        pism_version = 'no'
        picop_opt = 'yes'
        mparam = 'boxes_'+str(nD_config)+'_pism'+pism_version+'_picop'+picop_opt
              
        C = tuned_box_Cs.sel(config=nD_config, pism=pism_version, profile_domain=50).drop('config').drop('pism').drop('profile_domain')
        gamma = tuned_box_gammas.sel(config=nD_config, pism=pism_version, profile_domain=50).drop('config').drop('pism').drop('profile_domain')
                    
                        
        eval_1D = meltf.calculate_melt_rate_Gt_and_box1_all_isf(nisf_list, T_S_profile, 
                                                                 geometry_info_2D, geometry_info_1D, isf_stack_mask, 
                                                                 mparam, gamma, C=C, angle_option='appenB',
                                                                 box_charac_2D=box_charac_all_2D, box_charac_1D=box_charac_all_1D, box_tot=nD_config, 
                                                                 box_tot_option='nD_config', pism_version=pism_version, picop_opt='no',
                                                                 verbose=False)
        
        eval_1D.to_netcdf(outputpath_melt+'eval_metrics_1D_'+mparam+'_oneFRIS_'+str(yyear)+'.nc')        
                    


In [ ]:
yyear